In [1]:
import pandas as pd
import numpy as np
import collections
import time
import math

In [2]:
#                            ,usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties',
#                                    'Satisfactory Rating', 'Shower','Washing Machine', 'Fridge','3 Star',
#                                     'Air Conditioning','Good Rating','Car Park' ]
item_metadata = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\item_metadata_all.csv',
                           sep=','
                           ,usecols=['item_id','n_clicks','views','clicks_views_ratio','sum_of_properties',
                                   'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                                    '1 Star','2 Star','3 Star','4 Star','5 Star']
                           )
item_metadata['item_id'] = item_metadata['item_id'].astype(int)
item_metadata.shape

(813442, 14)

In [3]:
train = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\train.csv',
                       sep=',',
                    skiprows=range(1, 6000000)
#                    nrows=100000
                   )
train = train[train["action_type"] == "clickout item"]
print(train.shape)
train[:3]

(660653, 13)


,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
15,6000014,de86377f72f0f,1541248401,V2M2533G2TVI,7,clickout item,4936470,AU,"Melbourne, Australia",desktop,Serviced Apartment,109094|55074|101988|8472964|109095|848971|2291...,139|130|91|266|231|122|125|108|161|120|104|106...
34,6000033,0d993d2278335,1541248402,388RD30GUJBJ,3,clickout item,47173,UK,"Eastbourne, United Kingdom",mobile,NaN,47802|1062872|47173|1556531|46676|914675|32280...,148|63|76|71|50|44|63|59|80|71|82|50|62|57|64|...
37,6000036,d02ea044226bc,1541248402,F3W2KEFHJ2QL,16,clickout item,2326194,AR,"Maceió, Brazil",desktop,NaN,2326194|109324|2179366|1321131|111530|478651|2...,226|207|143|185|137|171|135|121|205|83|176|149...


In [4]:
def string_to_array(s):
    """Convert pipe separated string to array."""

    if isinstance(s, str):
        out = s.split("|")
    elif math.isnan(s):
        out = []
    else:
        raise ValueError("Value must be either string of nan")
    return out

def explode_impressions_and_prices(df_in):
    """Explode column col_expl of array type into multiple rows."""

    df = df_in.copy()
    df.loc[:, 'impressions'] = df['impressions'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]
    df.loc[:, 'prices'] = df['prices'].apply(string_to_array)  # zamienia 1|2|3 na [1,2,3]

    df = df.sort_values(by=['session_id'])
    df['number_of_impressions'] = df['impressions'].apply(len)
    
    df_out = pd.DataFrame(
        {col: np.repeat(df[col].values,
                        df['impressions'].str.len())
         for col in df.columns.drop('impressions')}
    )
    

    df_out.loc[:, 'impressions'] = np.concatenate(df['impressions'].values)
    df_out.loc[:, 'impressions'] = df_out['impressions'].apply(int)
    
    df_out.loc[:, 'prices'] = np.concatenate(df['prices'].values)
    df_out.loc[:, 'prices'] = df_out['prices'].apply(int)

    return df_out

In [5]:
train = explode_impressions_and_prices(train)
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126


In [6]:
train['position'] = train.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3


In [7]:
# # DODANIE CITY I COUNTRY
# city, country = (train['city']
#                     .values[0]
#                     .replace(" ", "")
#                     .split(','))
# print(city)
# print(country)

# cities = []
# countries = []
# l = []
# for index in range(len(train['city'].values)):
#     l = (train['city']
#                     .values[index]
#                     .replace(" ", "")
#                     .split(','))
#     cities.append(l[0])
#     countries.append(l[1])
# #     if len(l) > 2 :
# #         print(l[0])
# #         print(l[1])
# #         print(l[2])
# #         print(len(l))

# train = train.drop(['city'], axis=1)
# train['city'] = cities
# train['country'] = countries
# train[:3]

In [8]:
def mean_price(df):
    return df['prices'].mean()
df_mean_price = train.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
train = train.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
train['mean_price_difference'] = abs(train['prices'] - train['mean_price'])
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,mean_price,mean_price_difference
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,96,25,5479306,1,82.6,13.4
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,81,25,97126,2,82.6,1.6
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,Hotel|Resort,87,25,2195126,3,82.6,4.4


In [9]:
train = train.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
train[:3]

,index,session_id,timestamp,user_id,step,action_type,reference,platform,city,device,...,1 Star,2 Star,Satisfactory Rating,4 Star,Excellent Rating,Very Good Rating,Good Rating,5 Star,3 Star,sum_of_properties
0,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,89.0
1,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,72.0
2,8312303,000066611146f,1541332330,XRIOQ0R1CHLT,3,clickout item,5479306,BE,"Obaköy, Turkey",desktop,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,75.0


In [10]:
train = train.rename({'index':'item_id'})
train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position','mean_price','mean_price_difference',
               'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                '1 Star','2 Star','3 Star','4 Star','5 Star',
              ]]
# train = train[["item_id", "reference",'n_clicks','views','clicks_views_ratio','sum_of_properties','mean_price','mean_price_difference','position']]
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,mean_price,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star
0,5479306,5479306,2.0,38,0.052632,1,82.6,13.4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,97126,5479306,23.0,304,0.075658,2,82.6,1.6,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2195126,5479306,20.0,179,0.111732,3,82.6,4.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [11]:
train['reference'] = train['reference'].astype(int)
train['clicked'] = np.where(train['item_id'] == train['reference'], 1, 0)
train[:3]

,item_id,reference,n_clicks,views,clicks_views_ratio,position,mean_price,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked
0,5479306,5479306,2.0,38,0.052632,1,82.6,13.4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,97126,5479306,23.0,304,0.075658,2,82.6,1.6,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,2195126,5479306,20.0,179,0.111732,3,82.6,4.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [158]:
# usuwa zduplikowane hotele
# train.drop_duplicates('item_id', inplace=True)
# collections.Counter(train['item_id'].values)

In [159]:
# collections.Counter(train['clicked'].values)

In [160]:
# import plotly.express as px
# fig = px.scatter(train[:10000],x='n_clicks',y='views', color='clicked')
# fig.show()

In [12]:
train = train.drop(columns=['reference','mean_price'])
train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked
0,5479306,2.0,38,0.052632,1,13.4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,97126,23.0,304,0.075658,2,1.6,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,2195126,20.0,179,0.111732,3,4.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [13]:
# train['Rating'] = train['Satisfactory Rating']*1 + train['Good Rating']*2 + train['Very Good Rating']*3+train['Excellent Rating']*4
train['Stars'] = train['1 Star']*1 + train['2 Star']*2 + train['3 Star']*3 + train['4 Star']*4 + train['5 Star']*5

train['Rating'] = 0
train.loc[train['Satisfactory Rating']==1, 'Rating'] = 1
train.loc[train['Good Rating']==1, 'Rating'] = 2
train.loc[train['Very Good Rating']==1, 'Rating'] = 3
train.loc[train['Excellent Rating']==1, 'Rating'] = 4


train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked,Stars,Rating
0,5479306,2.0,38,0.052632,1,13.4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,5.0,3
1,97126,23.0,304,0.075658,2,1.6,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,4.0,3
2,2195126,20.0,179,0.111732,3,4.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,5.0,2


# Dane

In [107]:
# train.loc[train['clicked']==1][['Rating','Stars']].mean()

Rating    4.779974
Stars     2.346078
dtype: float64

In [96]:
import plotly.graph_objects as go
di = dict(collections.Counter(train.loc[train['clicked']==1]['Stars']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:6], x=list(di.keys())[:6])],
    layout_title_text="Liczba gwiazdek dla kliknięć"
)
fig.show()

In [98]:
di = dict(collections.Counter(train.loc[train['clicked']==0]['Stars']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:6], x=list(di.keys())[:6])],
    layout_title_text="Liczba gwiazdek dla nieklikniętych"
)
fig.show()

In [114]:
di = dict(collections.Counter(train.loc[train['clicked']==1]['Rating']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:5], x=list(di.keys())[:5])],
    layout_title_text="Liczba ratingu dla kliknięć"
)
fig.show()

In [115]:
di = dict(collections.Counter(train.loc[train['clicked']==0]['Rating']))
fig = go.Figure(
    data=[go.Bar(y=list(di.values())[:5], x=list(di.keys())[:5])],
    layout_title_text="Liczba ratingu dla nieklikniętych"
)
fig.show()

In [117]:
# import plotly.express as px
# fig = px.scatter(train[:1000],x='Rating',y='Stars', color='clicked')
# fig.show()

# Sieć neuronowa

In [78]:
# 'Satisfactory Rating','Shower','Washing Machine', 'Fridge','3 Star','Air Conditioning','Good Rating','Car Park'
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
X_train[:3]

,mean_price_difference,n_clicks,clicks_views_ratio,position,Rating,Stars
9382235,1.16,4.0,0.018957,10,4,4.0
8616454,18.96,45.0,0.145161,19,2,3.0
14949364,19.75,3.0,0.028846,17,3,0.0


In [79]:
# X_train_id_columns = X_train.reset_index().drop(columns=['index'], axis=1)['item_id']
X_train_rest_columns = X_train.reset_index().drop(columns=['index'], axis=1)[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']]

In [120]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train_rest_columns)
X_train_rest_columns = sc.transform(X_train_rest_columns)
# X_test = sc.transform(X_test)
X_train_rest_columns[:3]

array([[-0.2466967 , -1.46860439,  0.17398022,  0.47809154,  1.0241627 ],
       [-0.02016323,  1.18650325, -0.33071524, -0.26204823, -0.68071524],
       [ 0.548739  ,  1.46598827,  0.01990862,  1.21823131, -0.11242259]])

In [80]:
X_train = X_train_rest_columns

In [121]:
X_train = np.hstack((X_train_id_columns.values.reshape(-1,1), X_train_rest_columns)) # .values.reshape(-1,1)
X_train

array([[ 4.08940000e+04, -2.46696701e-01, -1.46860439e+00,
         1.73980225e-01,  4.78091539e-01,  1.02416270e+00],
       [ 4.67250000e+04, -2.01632268e-02,  1.18650325e+00,
        -3.30715237e-01, -2.62048233e-01, -6.80715243e-01],
       [ 1.67580000e+04,  5.48738997e-01,  1.46598827e+00,
         1.99086201e-02,  1.21823131e+00, -1.12422594e-01],
       ...,
       [ 1.04195800e+06,  3.25050466e+00, -1.46860439e+00,
         4.01065443e-01,  1.21823131e+00,  1.02416270e+00],
       [ 4.13859400e+06, -5.27809957e-01, -7.11793136e-02,
        -4.53974510e-02,  1.21823131e+00,  1.59245535e+00],
       [ 6.75773400e+06,  2.95470783e+00,  1.32624576e+00,
        -2.20075340e-01, -1.74232778e+00, -1.24900789e+00]])

In [81]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [83]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [85]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=6))
# model.add(Dense(70, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])
history = model.fit(X_train, y_train.values, epochs=3, batch_size=100)

Epoch 1/3
12140017/12140017 [==============================] - 127s 10us/step - loss: 0.1480 - accuracy: 0.9568 - f1_m: 0.0888 - precision_m: 0.2082 - recall_m: 0.0613
Epoch 2/3
12140017/12140017 [==============================] - 124s 10us/step - loss: 0.1430 - accuracy: 0.9574 - f1_m: 0.0839 - precision_m: 0.2031 - recall_m: 0.0568
Epoch 3/3
12140017/12140017 [==============================] - 125s 10us/step - loss: 0.1427 - accuracy: 0.9575 - f1_m: 0.0854 - precision_m: 0.2059 - recall_m: 0.0579


In [228]:
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=1)


1230908/1230908 [==============================] - 19s 15us/step


# NAUKA

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler, normalize
from imblearn.under_sampling import RandomUnderSampler

Using TensorFlow backend.


In [15]:
nans = lambda df: df[df.isnull().any(axis=1)]
print(len(train))
train = train.dropna()
print(len(train))

15180725
15175022


In [16]:
train[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,clicked,Stars,Rating
0,5479306,2.0,38,0.052632,1,13.4,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,5.0,3
1,97126,23.0,304,0.075658,2,1.6,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,4.0,3
2,2195126,20.0,179,0.111732,3,4.4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,5.0,2


In [45]:
# 'n_clicks','views','clicks_views_ratio', 'mean_price','mean_price_difference','position'
X_train, X_test, y_train, y_test = train_test_split(train[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']], 
                                                    train['clicked'],
                                                    test_size=0.2,
                                                    random_state=0)
# rus = RandomUnderSampler(random_state=0)
# X_train, y_train = rus.fit_resample(X_train, y_train)

In [46]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
# X_train = normalize(X_train, norm='l2')
X_test = sc.transform(X_test)
# X_test = normalize(X_test, norm='l2')

In [47]:
collections.Counter(y_train)

Counter({0: 11611618, 1: 528399})

In [48]:
from sklearn.linear_model import LogisticRegression
# logreg = DecisionTreeClassifier(class_weight={0.0:0.8, 1.0:0.2})
logreg = LogisticRegression(n_jobs=-1, class_weight={0.0:0.2, 1.0:0.8}) #class_weight={0.0:0.2, 1.0:0.8}
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
# y_pred_prob = logreg.predict_proba(X_test)

D:\Instalowane\Programy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Instalowane\Programy\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [49]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, balanced_accuracy_score

print(collections.Counter(y_test))
print(collections.Counter(y_pred))

print('balanced_accuracy_score: {0}'.format(balanced_accuracy_score(y_test, y_pred)))
print('accuracy_score: {0}'.format(accuracy_score(y_test, y_pred)))
print('Nieprawidłowo sklasyfikowane próbki: %d' % (y_test != y_pred).sum())
print('classification_report :\n', classification_report(y_test, y_pred))
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

Counter({0: 2903372, 1: 131633})
Counter({0: 2961102, 1: 73903})
balanced_accuracy_score: 0.5802814814078417
accuracy_score: 0.9477140894331311
Nieprawidłowo sklasyfikowane próbki: 158688
classification_report :
               precision    recall  f1-score   support

           0       0.96      0.98      0.97   2903372
           1       0.32      0.18      0.23    131633

    accuracy                           0.95   3035005
   macro avg       0.64      0.58      0.60   3035005
weighted avg       0.94      0.95      0.94   3035005

[[2852893   50479]
 [ 108209   23424]]


# ZBIÓR TESTOWY

In [69]:
test = pd.read_csv('D:\\Dokumenty\\Systemy_rekomendacyjne\\data\\test.csv',
                       sep=',',
#                     skiprows=range(1, 11000000)
#                    nrows=10000
                   )
# test = test[test["action_type"] == "clickout item" & test['reference'].isnull()]
mask = test["reference"].isnull() & (test["action_type"] == "clickout item")
test = test[mask]
# print(test.shape)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,impressions,prices
9,U9MAV55LZMOR,1541453209,0dd5544c71728,1,clickout item,NaN,US,"Atlantic City, USA",mobile,NaN,366186|56563|3741160|63568|3478738|3137008|678...,69|68|46|39|69|47|48|52|74|70|44|34|57|69|37|7...
15,VXIWZ3NF0WHF,1541453212,be1b5623b2e12,3,clickout item,NaN,DK,"Copenhagen, Denmark",tablet,NaN,4412788|53876|3147220|929429|148127|53835|1753...,122|144|195|100|92|117|72|107|268|166|171|657|...
16,2VZWKE7NWP8C,1541453213,7daf68934bfef,1,clickout item,NaN,US,"Weston, USA",mobile,NaN,57189|404846|164156|111309|148435|77409|103228...,115|166|115|97|111|87|200|101|114|160|98|97|10...


In [70]:
test = explode_impressions_and_prices(test)
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016


In [52]:
test['position'] = test.groupby(['user_id','timestamp','session_id','step']).cumcount()+1
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,1
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,2
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,3


In [53]:
index_df = test[["user_id","session_id","timestamp","step","impressions"]]
index_df[:10]

,user_id,session_id,timestamp,step,impressions
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717


In [54]:
df_mean_price = test.groupby(['session_id', 'timestamp']).apply(lambda df: df['prices'].mean()).to_frame("mean_price")
df_mean_price[:3]
test = test.merge(df_mean_price,
                     left_on=['session_id', 'timestamp'],
                     right_on=['session_id', 'timestamp'],
                     how='left')
test['mean_price_difference'] = abs(test['prices'] - test['mean_price'])
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,prices,number_of_impressions,impressions,position,mean_price,mean_price_difference
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,187,25,3811810,1,58.04,128.96
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,78,25,496141,2,58.04,19.96
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,100,25,674016,3,58.04,41.96


In [55]:
test = test.merge(item_metadata,
                   left_on='impressions',
                   right_on='item_id',
                   how='left')
test[:3]

,user_id,timestamp,session_id,step,action_type,reference,platform,city,device,current_filters,...,1 Star,2 Star,Satisfactory Rating,4 Star,Excellent Rating,Very Good Rating,Good Rating,5 Star,3 Star,sum_of_properties
0,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,48.0
1,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,76.0
2,0TBXPQCK401O,1541484101,000056cd97ce2,4,clickout item,NaN,MY,"Port Dickson, Malaysia",desktop,NaN,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,46.0


In [56]:
test.columns

Index(['user_id', 'timestamp', 'session_id', 'step', 'action_type',
       'reference', 'platform', 'city', 'device', 'current_filters', 'prices',
       'number_of_impressions', 'impressions', 'position', 'mean_price',
       'mean_price_difference', 'item_id', 'n_clicks', 'views',
       'clicks_views_ratio', '1 Star', '2 Star', 'Satisfactory Rating',
       '4 Star', 'Excellent Rating', 'Very Good Rating', 'Good Rating',
       '5 Star', '3 Star', 'sum_of_properties'],
      dtype='object')

In [57]:
train.columns

Index(['item_id', 'n_clicks', 'views', 'clicks_views_ratio', 'position',
       'mean_price_difference', 'Satisfactory Rating', 'Good Rating',
       'Very Good Rating', 'Excellent Rating', '1 Star', '2 Star', '3 Star',
       '4 Star', '5 Star', 'clicked', 'Stars', 'Rating'],
      dtype='object')

In [58]:
test = test.rename({'index':'item_id'})
test = test[["item_id", "reference",'n_clicks','views','clicks_views_ratio','position', 'mean_price_difference',
               'Satisfactory Rating','Good Rating','Very Good Rating', 'Excellent Rating',
                '1 Star','2 Star','3 Star','4 Star','5 Star',
              ]]
test = test.drop(columns=['reference'], axis=1)
test = test.dropna()
test[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star
0,3811810.0,238.0,1946.0,0.122302,1,128.96,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,496141.0,258.0,1951.0,0.132240,2,19.96,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,674016.0,93.0,2184.0,0.042582,3,41.96,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [59]:
test['Stars'] = test['1 Star']*1 + test['2 Star']*2 + test['3 Star']*3 + test['4 Star']*4 + test['5 Star']*5

test['Rating'] = 0
test.loc[test['Satisfactory Rating']==1, 'Rating'] = 1
test.loc[test['Good Rating']==1, 'Rating'] = 2
test.loc[test['Very Good Rating']==1, 'Rating'] = 3
test.loc[test['Excellent Rating']==1, 'Rating'] = 4


test[:3]

,item_id,n_clicks,views,clicks_views_ratio,position,mean_price_difference,Satisfactory Rating,Good Rating,Very Good Rating,Excellent Rating,1 Star,2 Star,3 Star,4 Star,5 Star,Stars,Rating
0,3811810.0,238.0,1946.0,0.122302,1,128.96,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,3
1,496141.0,258.0,1951.0,0.132240,2,19.96,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,2
2,674016.0,93.0,2184.0,0.042582,3,41.96,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,2


In [60]:
X_test = test[['mean_price_difference','n_clicks','clicks_views_ratio','position','Rating','Stars']]
X_test = sc.transform(X_test)

In [61]:
y_pred = logreg.predict_proba(X_test)

In [62]:
y_pred

array([[0.4691166 , 0.5308834 ],
       [0.46852522, 0.53147478],
       [0.74165413, 0.25834587],
       ...,
       [0.98193593, 0.01806407],
       [0.9730417 , 0.0269583 ],
       [0.96771327, 0.03228673]])

In [63]:
result = pd.concat([index_df, pd.DataFrame(y_pred)], axis=1, sort=False)
# result = result.drop([0], axis=1)
result[:10]

,user_id,session_id,timestamp,step,impressions,0,1
0,0TBXPQCK401O,000056cd97ce2,1541484101,4,3811810,0.469117,0.530883
1,0TBXPQCK401O,000056cd97ce2,1541484101,4,496141,0.468525,0.531475
2,0TBXPQCK401O,000056cd97ce2,1541484101,4,674016,0.741654,0.258346
3,0TBXPQCK401O,000056cd97ce2,1541484101,4,1153426,0.588223,0.411777
4,0TBXPQCK401O,000056cd97ce2,1541484101,4,1339848,0.823942,0.176058
5,0TBXPQCK401O,000056cd97ce2,1541484101,4,8400372,0.820463,0.179537
6,0TBXPQCK401O,000056cd97ce2,1541484101,4,693816,0.821667,0.178333
7,0TBXPQCK401O,000056cd97ce2,1541484101,4,521336,0.863746,0.136254
8,0TBXPQCK401O,000056cd97ce2,1541484101,4,519061,0.858224,0.141776
9,0TBXPQCK401O,000056cd97ce2,1541484101,4,1482717,0.899583,0.100417


In [64]:
result = result.assign(impressions=lambda x: x["impressions"].apply(str))
result = result.sort_values(["user_id", "session_id", "timestamp", "step", 1],
                     ascending=[True, True, True, True, False])
result[:10]

,user_id,session_id,timestamp,step,impressions,0,1
2958350,000I430EXZC0,ddabdf53a4d38,1541501642,1,98265,0.127260,0.872740
2958356,000I430EXZC0,ddabdf53a4d38,1541501642,1,2111996,0.610007,0.389993
2958365,000I430EXZC0,ddabdf53a4d38,1541501642,1,103589,0.662377,0.337623
2958347,000I430EXZC0,ddabdf53a4d38,1541501642,1,8805412,0.686129,0.313871
2958357,000I430EXZC0,ddabdf53a4d38,1541501642,1,2350650,0.689581,0.310419
2958358,000I430EXZC0,ddabdf53a4d38,1541501642,1,3143731,0.732434,0.267566
2958348,000I430EXZC0,ddabdf53a4d38,1541501642,1,103452,0.734524,0.265476
2958353,000I430EXZC0,ddabdf53a4d38,1541501642,1,321771,0.747349,0.252651
2958354,000I430EXZC0,ddabdf53a4d38,1541501642,1,1148728,0.760344,0.239656
2958359,000I430EXZC0,ddabdf53a4d38,1541501642,1,103495,0.770523,0.229477


In [65]:
result.dtypes

user_id         object
session_id      object
timestamp        int64
step             int64
impressions     object
0              float64
1              float64
dtype: object

In [66]:
def group_concat(df, gr_cols, col_concat):
    """Concatenate multiple rows into one."""

    df_out = (
        df
        .groupby(gr_cols)[col_concat]
        .apply(lambda x: ' '.join(x))
        .to_frame()
        .reset_index()
    )
    return df_out

In [76]:
result = group_concat(result, ["user_id","session_id","timestamp","step"], "impressions")
result = result.rename(columns={"impressions":"item_recommendations"})
result[:10]

,user_id,session_id,timestamp,step,item_recommendations
0,000I430EXZC0,ddabdf53a4d38,1541501642,1,3247218 8805412 103452 95011 98265 103658 1036...
1,000O8CPC6T0W,846b712ae8e1f,1541480658,1,15196 923037 52196 51577 53603 51321 13876 528...
2,000REY2YYH1D,4a313f2eb8954,1541465923,1,65491 67551 70266 1045348 62474 65392 57541 68...
3,0010PD3FH7SV,26b9e7e18cc87,1541535269,2,5963554 2267078 3443066 1207371 2826362 383066...
4,0012IHLNTQWC,a035e580f0e64,1541505846,39,2041695 1326566 3171798 1100198 1069656 106965...
5,0015CRBC6OH9,8bcbd478ac2cf,1541519175,3,11926 82322 48827 48847 48405 48816 50434 1208...
6,0018EAWF0OIK,10ce4a98a476f,1541513087,2,998597 2567604 998593 8230426 4072668 3804510 ...
7,001EBIZM062K,0e036884adc53,1541502068,1,214161 2467696 2164460 42278 9616040 3154396 9...
8,001G88F8NP9P,2a2e9eb9c967c,1541511909,9,400791 1306503 972027 1314825 2855876 2678882 ...
9,001JFV0AYE6Z,73ad1e08b4430,1541481561,1,67196 61991 149620 59822 59397 58414 6002482 7...


In [77]:
result.to_csv("D:\\Dokumenty\\Systemy_rekomendacyjne\\additional_resources\\2019-master\\data\\submission_ML.csv", index=False,)